# Rocket Flight Simulation with RK4, 2D 3DOF, and Pyatmos

This notebook simulates a rocket flight in two dimensions using the RK4 integration scheme. It accounts for:

- A thrust curve loaded from a folder of predefined CSV files. Each CSV file contains two columns: time (s) and thrust (N). The file name (minus `.csv`) is used as the motor name.
- Variable rocket parameters (mass, geometry, etc.)
- Drag (using the drag equation with user-specified diameter, drag coefficient, etc.)
- A standard atmosphere model from **pyatmos** (for density, temperature, and speed of sound vs altitude)
- Wind effects and a fixed launch tilt angle

The state vector is defined as:

$
\mathbf{x}(t) = \begin{bmatrix} x(t) \\ y(t) \\ v_x(t) \\ v_y(t) \\ m(t) \end{bmatrix}
$

and the equations of motion are given in the notebook. The interactive UI lets the user choose a motor and customize the rocket parameters.

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import glob
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display

try:
    import pyatmos
except ImportError:
    raise ImportError("pyatmos is required. Please install it via pip (pip install pyatmos)")

g = 9.80665

def load_thrust_curve_from_csv(filename):
    data = pd.read_csv(filename)
    return data['Time (s)'].values, data[' Thrust (N)'].values

motor_files = glob.glob("*.csv")
motors = {}
for file in motor_files:
    motor_name = file.replace(".csv", "")
    t_time, t_values = load_thrust_curve_from_csv(file)
    if t_time is not None:
        thrust_func = interp1d(t_time, t_values, fill_value=0, bounds_error=False)
        motors[motor_name] = {
            'thrust_time': t_time,
            'thrust_values': t_values,
            'thrust_func': thrust_func,
            'burn_time': t_time[-1],
            'Isp': 250
        }

if not motors:
    print("No motor CSV files found in the current directory. Please add your thrust curve CSV files.")

def atmosphere_properties(altitude):
    props = pyatmos.coesa76(altitude / 1000)
    a = np.sqrt(1.4 * 287 * props.T)
    return props.rho, props.T, a

def rk4_step(f, t, y, dt, *args):
    k1 = f(t, y, *args)
    k2 = f(t + dt/2, y + dt*k1/2, *args)
    k3 = f(t + dt/2, y + dt*k2/2, *args)
    k4 = f(t + dt, y + dt*k3, *args)
    return y + dt*(k1 + 2*k2 + 2*k3 + k4)/6

def dynamics(t, state, motor, rocket_params, sim_params):
    x, y, vx, vy, m = state

    if t <= motor['burn_time']:
        T = motor['thrust_func'](t)
        dm_dt = -rocket_params['propellant_mass'] / motor['burn_time']
    else:
        T = 0
        dm_dt = 0

    theta_rad = np.deg2rad(rocket_params['launch_angle'])
    T_x = T * np.sin(theta_rad)
    T_y = T * np.cos(theta_rad)

    v_wx = sim_params['wind']
    v_rel = np.array([vx - v_wx, vy])
    v_rel_mag = np.linalg.norm(v_rel)

    dens, temp, a_sound = atmosphere_properties(max(y, 0))
    A = np.pi * (rocket_params['diameter'] / 2)**2
    Cd = rocket_params['Cd']
    D = -0.5 * dens * Cd * A * v_rel_mag**2 * (v_rel / v_rel_mag) if v_rel_mag > 0 else np.array([0.0, 0.0])

    ax = (T_x + D[0]) / m
    ay = (T_y + D[1]) / m - g

    return np.array([vx, vy, ax, ay, dm_dt])

def run_simulation(motor, rocket_params, sim_params, dt=0.01):
    initial_mass = rocket_params['dry_mass'] + rocket_params['propellant_mass']
    state = np.array([0.0, 0.0, rocket_params['v0_x'], rocket_params['v0_y'], initial_mass])

    results = {
        'time': [], 'x': [], 'y': [], 'vx': [], 'vy': [], 'mass': [],
        'drag': [], 'dynamic_pressure': [], 'mach': [], 'vertical_velocity': [],
        'acceleration': [], 'temperature': [], 'speed_of_sound': [],
        'ke': [], 'pe': [], 'total_energy': [], 'stag_temp': []
    }

    t = 0
    while True:
        x, y, vx, vy, m = state

        results['time'].append(t)
        results['x'].append(x)
        results['y'].append(max(y, 0))
        results['vx'].append(vx)
        results['vy'].append(vy)
        results['mass'].append(m)
        results['vertical_velocity'].append(vy)

        dens, temp, a_sound = atmosphere_properties(max(y, 0))
        results['temperature'].append(temp)
        results['speed_of_sound'].append(a_sound)

        v_rel = np.array([vx - sim_params['wind'], vy])
        v_rel_mag = np.linalg.norm(v_rel)

        A = np.pi * (rocket_params['diameter'] / 2)**2
        drag = 0.5 * dens * rocket_params['Cd'] * A * v_rel_mag**2
        results['drag'].append(drag)

        q = 0.5 * dens * v_rel_mag**2
        results['dynamic_pressure'].append(q)

        mach = v_rel_mag / a_sound if a_sound > 0 else 0
        results['mach'].append(mach)

        ke = 0.5 * m * (vx**2 + vy**2)
        pe = m * g * y
        results['ke'].append(ke)
        results['pe'].append(pe)
        results['total_energy'].append(ke + pe)

        stag_temp = temp * (1 + 0.2 * mach**2)
        results['stag_temp'].append(stag_temp)

        deriv = dynamics(t, state, motor, rocket_params, sim_params)
        acc = deriv[3]
        results['acceleration'].append(acc)

        if y <= 0 and vy <= 0 and t > 0:
            break

        state = rk4_step(dynamics, t, state, dt, motor, rocket_params, sim_params)
        t += dt

    for key in results:
        results[key] = np.array(results[key])

    print("\n--- Flight Statistics ---")
    apogee = np.max(results['y'])
    time_to_apogee = results['time'][np.argmax(results['y'])]
    max_q = np.max(results['dynamic_pressure'])
    max_speed = np.max(np.sqrt(results['vx']**2 + results['vy']**2))
    max_mach = np.max(results['mach'])

    print(f"Apogee: {apogee:.2f} m")
    print(f"Time to Apogee: {time_to_apogee:.2f} s")
    print(f"Max Q (dynamic pressure): {max_q:.2f} Pa")
    print(f"Max Speed: {max_speed:.2f} m/s")
    print(f"Max Mach Number: {max_mach:.2f}")

    return results

def plot_results(results, motor_burn_time=None):
    time = results['time']
    alt = results['y']

    impact_time = time[-1]
    burnout_time = motor_burn_time if motor_burn_time is not None else 0

    def draw_event_lines(ax):
        if burnout_time:
            ax.axvline(x=burnout_time, color='orange', linestyle='--', label='Burnout')
        ax.axvline(x=impact_time, color='red', linestyle='--', label='Impact')
        ax.legend()

    plt.figure(figsize=(14, 10))

    for i, (title, ylabel, data) in enumerate([
        ('Altitude vs Time', 'Altitude (m)', alt),
        ('Vertical Velocity vs Time', 'Vertical Velocity (m/s)', results['vertical_velocity']),
        ('Drag vs Time', 'Drag (N)', results['drag']),
        ('Mass vs Time', 'Mass (kg)', results['mass']),
        ('Dynamic Pressure vs Time', 'Dynamic Pressure (Pa)', results['dynamic_pressure']),
        ('Mach Number vs Time', 'Mach Number', results['mach']),
        ('Vertical Acceleration vs Time', 'Vertical Acceleration (m/s²)', results['acceleration'])
    ]):
        plt.subplot(3, 3, i + 1)
        plt.plot(time, data)
        plt.xlabel('Time (s)')
        plt.ylabel(ylabel)
        plt.title(title)
        plt.grid(True)
        draw_event_lines(plt.gca())

    # Energy plot
    plt.subplot(3, 3, 8)
    plt.plot(time, results['ke'], label='Kinetic Energy')
    plt.plot(time, results['pe'], label='Potential Energy')
    plt.plot(time, results['total_energy'], label='Total Energy')
    plt.xlabel('Time (s)')
    plt.ylabel('Energy (J)')
    plt.title('Energy vs Time')
    plt.legend()
    plt.grid(True)
    draw_event_lines(plt.gca())

    # Stagnation temp plot
    plt.subplot(3, 3, 9)
    plt.plot(time, results['stag_temp'])
    plt.xlabel('Time (s)')
    plt.ylabel('Stag Temp (K)')
    plt.title('Stagnation Temperature vs Time')
    plt.grid(True)
    draw_event_lines(plt.gca())

    plt.tight_layout()
    plt.show()

def run_interface():
    motor_dropdown = widgets.Dropdown(
        options=list(motors.keys()),
        value=list(motors.keys())[0] if motors else None,
        description='Motor:'
    )
    rail_length_text = widgets.FloatText(value=1.5, description='Rail Length (m):')
    wind_text = widgets.FloatText(value=0.0, description='Wind (m/s):')
    diameter_text = widgets.FloatText(value=98, description='Diameter (mm):')
    Cd_text = widgets.FloatText(value=0.75, description='Cd:')
    dry_mass_text = widgets.FloatText(value=10.0, description='Dry Mass (kg):')
    propellant_mass_text = widgets.FloatText(value=5.0, description='Propellant Mass (kg):')
    run_button = widgets.Button(description='Run Simulation')
    out = widgets.Output()

    def on_run_button_clicked(b):
        with out:
            out.clear_output()
            selected_motor = motors[motor_dropdown.value]
            rocket_params = {
                'diameter': diameter_text.value / 1000,
                'Cd': Cd_text.value,
                'dry_mass': dry_mass_text.value,
                'propellant_mass': propellant_mass_text.value
            }
            sim_params = {'wind': wind_text.value}

            print(f"Running simulation with {motor_dropdown.value}...")

            # Pre-rail simulation
            state = np.array([0.0, 0.0, 0.0, 0.0, rocket_params['dry_mass'] + rocket_params['propellant_mass']])
            t = 0
            dt = 0.001
            s = 0
            while s < rail_length_text.value and t < selected_motor['burn_time']:
                deriv = dynamics(t, state, selected_motor, {**rocket_params, 'launch_angle': 90}, sim_params)
                state = rk4_step(dynamics, t, state, dt, selected_motor, {**rocket_params, 'launch_angle': 90}, sim_params)
                s += state[3] * dt
                t += dt

            v_exit = np.linalg.norm(state[2:4])
            theta_launch_rad = np.arctan2(state[3], state[2] - sim_params['wind'])

            rocket_params['launch_angle'] = np.rad2deg(theta_launch_rad)
            rocket_params['v0_x'] = v_exit * np.sin(theta_launch_rad)
            rocket_params['v0_y'] = v_exit * np.cos(theta_launch_rad)

            results = run_simulation(selected_motor, rocket_params, sim_params, dt=0.01)
            plot_results(results, motor_burn_time=selected_motor['burn_time'])

    run_button.on_click(on_run_button_clicked)

    ui = VBox([
        HBox([motor_dropdown, rail_length_text, wind_text]),
        HBox([diameter_text, Cd_text, dry_mass_text, propellant_mass_text]),
        run_button,
        out
    ])

    display(ui)

if __name__ == "__main__":
    run_interface()
